In [134]:
import pandas as pd
import numpy as np

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

from collections import Counter
from collections import OrderedDict

import string
import re
import requests
import json

In [135]:
data = pd.read_json('../gg2015.json')
data = pd.DataFrame(data)
data = data['text'].values.tolist()

ValueError: Expected object or value

In [112]:
# return cleaned Tweet as string
# remove stopwords, user handles, punctuation, urls

def cleanTweets(tweet):
    tt = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=True)

    punctuation = list(string.punctuation)
    
    # strip stopwords, punctuation, url components 
    stop = stopwords.words('english') + punctuation + ['t.co', 'http', 'https', '...', '..', ':\\', 'RT', '#']

    strip_nums = re.sub("\d+", "", tweet)
    tokenized = tt.tokenize(strip_nums)
    terms_stop = [term for term in tokenized if term not in stop]
    cleaned = [term for term in terms_stop]
    cleaned = ' '.join(cleaned)
    
    return cleaned

In [113]:
data['text'] = data['text'].map(cleanTweets)

In [157]:
def findHosts(data):
    include_terms = ['host', 'hosted', 'hosting', 'hosts']
    remove_terms = ['next year']
    host = [];
    cohost = False;

    for tweet in data:
        if any(term in tweet for term in include_terms) and any(term not in tweet for term in remove_terms):
            host.append(tweet)
        if 'cohost' in tweet:
            cohost = True
            
    bgrams = [];

    for tweet in host:
        bgrams += list(nltk.bigrams(tweet.split()))
        
    fdist = nltk.FreqDist(bgrams)
    
    if cohost:
        fdist = fdist.most_common(100)
    else:
        fdist = fdist.most_common(1)
        
    final_hosts = []
    for host in fdist:
        name = host[0][0] + ' ' + host[0][1]
        final_hosts.append(name)
        
    return fdist

In [158]:
print(findHosts(data))

[(('Tina', 'Fey'), 585), (('Amy', 'Poehler'), 568), (('Fey', 'Amy'), 421), (('Tina', 'Amy'), 360), (('Golden', 'Globes'), 355), (('host', 'everything'), 167), (('Poehler', 'host'), 141), (('Poehler', 'Tina'), 132), (('Amy', 'Tina'), 128), (('Amy', 'host'), 120)]


In [199]:
# DATA PASSED IN AS LIST
def extractHosts(data, people_kb):
    # clean data
    cleaned_data = []
    
    for tweet in data:
        tt = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=True)

        punctuation = list(string.punctuation)

        # strip stopwords, punctuation, url components 
        stop = stopwords.words('english') + punctuation + ['t.co', 'http', 'https', '...', '..', ':\\', 'RT', '#']

        strip_nums = re.sub("\d+", "", tweet)
        tokenized = tt.tokenize(strip_nums)
        terms_stop = [term for term in tokenized if term not in stop]
        cleaned = [term for term in terms_stop]
        cleaned = ' '.join(cleaned)
        cleaned_data.append(cleaned)
     
    
    # find host
    include_terms = ['host', 'hosted', 'hosting', 'hosts']
    remove_terms = ['next year']
    host = [];
    cohost = False;

    for tweet in cleaned_data:
        if any(term in tweet for term in include_terms) and any(term not in tweet for term in remove_terms):
            host.append(tweet)
        if 'cohost' in tweet:
            cohost = True
            
    bgrams = [];

    for tweet in host:
        bgrams += list(nltk.bigrams(tweet.split()))
        
    fdist = nltk.FreqDist(bgrams)
    
    if cohost: #and len(final_hosts) > 1:
        fdist = fdist.most_common()
    else:
        fdist = fdist.most_common(1)
    
    final_hosts = []
    for host in fdist:
        name = host[0][0] + ' ' + host[0][1]
        final_hosts.append(name)
    
    return_list = []
    
    if cohost:
        for name in final_hosts:
            if name in people_kb:
                return_list.append(name)
        return_list = return_list[:2]
    else:
        return_list.append(final_hosts[0])
    
    return return_list
    

In [200]:
extractHosts(data,['Tina Fey'])

[(('Tina', 'Fey'), 585), (('Amy', 'Poehler'), 568), (('Fey', 'Amy'), 421), (('Tina', 'Amy'), 360), (('Golden', 'Globes'), 355)]



['Tina Fey']

In [202]:
final_hosts = ['Tina Fey', 'Amy Poehler', 'Fey Amy', 'Tina Amy', 'Golden Globes']
final_hosts = final_hosts[:2]
print(final_hosts)

['Tina Fey', 'Amy Poehler']
